In [1]:
import cv2
import numpy as np

In [19]:
imagepath = r"C:\Users\stefa\Desktop\Queens\Fourth Year\MTHE493\Database\E00\\"
image_1 = imagepath + "Frame055.png"           # your goal image
image_0 = imagepath + "Frame049.png"         # your initial region
redpic = imagepath + "Frame049red.png"
out_1 = imagepath + "Frame055.png"
stop_it = 20

# all these photos should be the same dimensions, and be square


def main():
    my_image_0 = cv2.imread(image_0)            # import original region image
    my_image_1 = cv2.imread(image_1)            # import destination region image
    my_image_red = cv2.imread(redpic)
    out_image1 = cv2.imread(out_1)            # import image to print on
    out_image = out_image1.copy()
    # my_image_0 = noisify(my_image_0)            # add noise to original region
    region_array, cellavg, outeravg, original_area, intensity, sq_intensity, comp_area, comp_intensity = region_definition(my_image_red, my_image_1)    # define array u based on original region image
    flag = 0
    thresh = int(0.005*original_area)
    print(thresh)
    print_photo(region_array, out_image, 0, flag)
    out_image = out_image1.copy()
    intensity_inv = invariance_of_intensity(my_image_1, cellavg, outeravg)
    i = 0
    new_area = original_area
    while i < 2500:                             # iterate through algorithm
        F_var = intensity_variance(my_image_1, new_area, cellavg, sq_intensity, intensity)
        F_mean = invariance_of_mean_intensity(my_image_1, new_area, comp_area, intensity, comp_intensity, cellavg, outeravg)
        f_matrix, F_max = functional(my_image_1, new_area, original_area, intensity_inv, F_var, F_mean)  # calculate functional values
        array_out, new_area, intensity, sq_intensity, comp_area, comp_intensity, flag = level_curves_algorithm(region_array, f_matrix, my_image_1, F_max, flag, thresh)
        print(flag)
        i += 1
        if i == 90:
            flag = stop_it
            print_photo(region_array, out_image, i, flag)
            break
        region_array = array_out.copy()
        print(i)
        if i % 10 == 0:            # print region every n steps, making n larger will make algorithm run slightly faster
            print_photo(region_array, out_image, i, flag)
            out_image = out_image1.copy()
    print_photo(region_array, out_image, i, flag)


def region_definition(image, image2):                   # function to define the array u based on original region
    # the definition of this region is currently specified to a black and white image. this will change when we move to
    # images of cells and define criteria used to identify region. We can use the function Stefan wrote here to define
    # the input region based on a drawn circle.
    cellpixels = []
    outerpixels = []
    intensity = 0
    sq_intensity = 0
    comp_area = 0
    comp_intensity = 0
    (height, width, x) = image.shape            # define array based on image shape + 2 for width and height
    array = [[-1 for i in range(height + 2)] for j in range(width + 2)]         # default values to -1
    for x in range(0, width):                   # iterate through image and check pixel values
        for y in range(0, height):
            if image[y, x][0] == 0 and image[y, x][1] == 0 and image[y, x][2] == 255:
                cellpixels.append((x, y))
                array[y + 1][x + 1] = 1
                intensity += image2[y][x][0]
                sq_intensity += image2[y][x][0]**2
            else:
                outerpixels.append((x, y))          # if pixel value is 0 (black) pixel is in region
                comp_intensity += image2[y][x][0]
    array[0][:] = array[1][:]                   # pad outer rows and columns of array
    array[height + 1][:] = array[height][:]
    array[:][0] = array[:][1]
    array[:][width + 1] = array[:][width]
    cellavg, outeravg = average_pixels(cellpixels, outerpixels)
    return array, cellavg, outeravg, len(cellpixels), intensity, sq_intensity, len(outerpixels), comp_intensity


def average_pixels(cellpixels, outerpixels):
    cellsum = 0
    outsum = 0
    img = cv2.imread(image_0)
    grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    for i in cellpixels:
        cellsum = cellsum + grayimg[i[1], i[0]]
    for i in outerpixels:
        outsum = outsum + grayimg[i[1], i[0]]
    cellavg = cellsum/len(cellpixels)
    outavg = outsum/len(outerpixels)
    return cellavg, outavg


def noisify(photo):                             # function to add noise to initial image
    (height, width, x) = photo.shape
    mean = 0                              # defining gaussian white noise, mean of zero, variance can change as desired
    var = 30
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, (height, width))
    for x in range(0, height):            # iterate through image array and add noise
        for y in range(0, width):         # if output values are outside of [0, 255], set it to the max or the min
            if photo[y][x][0] + gaussian[x, y] < 0:
                photo[y][x][0] = 0
            elif photo[y][x][0] + gaussian[x, y] > 255:
                photo[y][x][0] = 255
            else:
                photo[y][x][0] = photo[y][x][0] + gaussian[x, y]
    return photo


def functional(my_image, current_area, original_area, inv_int, F_var, F_mean):
    F_max = 0
    F1 = inv_int
    F2 = invariance_of_area(current_area, original_area)
    (height, width, x) = my_image.shape
    F = [[0 for i in range(height)] for j in range(width)]
    for x in range(0, width):
        for y in range(0, height):
            F[y][x] = 0.5*F1[y][x] + 2.5*F_mean[y][x] + 1.5*F2 + .1*F_var[y][x]
            if F[y][x] > F_max:
                F_max = F[y][x]
    return F, F_max


def invariance_of_area(current_area, original_area):
    F_area = 2*(current_area - original_area)
    return F_area


def invariance_of_intensity(my_image, cellavg, outeravg):      # function to define the functional of our region.
    (height, width, x) = my_image.shape
    F_inv = [[0 for i in range(height)] for j in range(width)]
    for x in range(0, width):
        for y in range(0, height):
            F_inv[y][x] = (int(my_image[y][x][0]) - cellavg)**2 - (int(my_image[y][x][0]) - outeravg)**2
    return F_inv


def invariance_of_mean_intensity(my_image, inner_area, outer_area, inner_sum, outer_sum, cellavg, outeravg):      # function to define the functional of our region.
    (height, width, x) = my_image.shape
    F_mean_inv = [[0 for i in range(height)] for j in range(width)]
    curr_inner = inner_sum / inner_area
    curr_outer = outer_sum / outer_area
    for x in range(0, width):
        for y in range(0, height):
            curr = int(my_image[y][x][0])
            F_mean_inv[y][x] = 2*(curr_inner - cellavg)*(curr - curr_inner)/inner_area + 2*(curr_outer - outeravg)*(-curr + curr_outer)/outer_area
    return F_mean_inv


def intensity_variance(my_image, current_area, cellavg, sq_intensity, intensity):
    (height, width, x) = my_image.shape
    F_var = [[0 for i in range(height)] for j in range(width)]
    sqa = sq_intensity / current_area
    ia = intensity / current_area
    for x in range(0, width):
        for y in range(0, height):
            curr = int(my_image[y][x][0])
            F_var[y][x] = 2*(sqa - ia**2 - cellavg)*((curr**2 - sqa)/current_area - 2*sqa*(curr - ia) / current_area)
    return F_var


def level_curves_algorithm(array1, f_matrix, my_image_1, F_max, flag, thresh):   # function to implement level curves algorithm as defined in
    count = 0
    area = 0
    comp_area = 0
    comp_intensity = 0
    print(F_max)
    epsilon = 10**(-100)
    lambd = 10
    delta_t = .75 / (F_max + epsilon)
    if F_max < 5:
        delta_t = 10**(-20)
        lambd = 1
    print(delta_t)
    height = len(array1[0])
    width = len(array1)
    intensity = 0
    sq_intensity = 0
    array2 = [[-1 for i in range(height)] for j in range(width)]
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            x1 = array1[y][x + 1]
            x2 = array1[y][x - 1]
            y1 = array1[y + 1][x]
            y2 = array1[y - 1][x]
            xy = array1[y][x]
            ux = (x1 - x2)/2
            uy = (y1 - y2)/2
            uxx = x1 - 2*xy + x2
            uyy = y1 - 2*xy + y2
            uxy = (array1[y + 1][x + 1] - array1[y + 1][x - 1] - array1[y - 1][x + 1] + array1[y - 1][x - 1])/4
            diff = (ux**2) + (uy**2)
            dpx = x1 - xy
            dmx = xy - x2
            dpy = y1 - xy
            dmy = xy - y2
            gradient_plus = (max(dmx, 0)**2 + min(dpx, 0)**2 + max(dmy, 0)**2 + min(dpy, 0)**2)**1/2
            gradient_minus = (max(dpx, 0)**2 + min(dmx, 0)**2 + max(dpy, 0)**2 + min(dmy, 0)**2)**1/2
            k = (uxx*(uy**2) - 2*uy*ux*uxy + uyy*(ux**2))/((diff + epsilon)**(3/2))
            new = xy + delta_t*(-(max(f_matrix[y - 1][x - 1], 0)*gradient_plus + min(f_matrix[y - 1][x - 1], 0)*gradient_minus) + lambd*k*(diff**(1/2)))
            if new > 1000000000000000000000000:
                new = 1000000000000000000000000
            elif new < -1000000000000000000000000:
                new = - 1000000000000000000000000
            array2[y][x] = new
            if new > 0:
                sq_intensity += my_image_1[y - 1][x - 1][0]**2
                intensity += my_image_1[y - 1][x - 1][0]
                area += 1
            else:
                comp_intensity += my_image_1[y - 1][x - 1][0]
                comp_area += 1
            if new * xy < 0:
                count += 1
    if count < thresh:
        flag = flag + 1
    else:
        flag = 0
    return array2, area, intensity, sq_intensity, comp_area, comp_intensity, flag

savepath = r"C:\Users\stefa\Desktop\Queens\Fourth Year\MTHE493\Database\E00\Results\\"

def print_photo(array, image, i, flag):           # print out resulting region, will need to update as we input functional
    height = len(array[0])
    width = len(array)
    image2 = image.copy()
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            if array[y][x]*array[y-1][x] < 0 or array[y][x]*array[y][x-1] < 0 or array[y][x]*array[y+1][x] < 0 or array[y][x]*array[y][x + 1] < 0:
                image2[y - 1, x - 1][0] = 0
                image2[y - 1, x - 1][1] = 0
                image2[y - 1, x - 1][2] = 255
    if i < 10:
        cv2.imwrite(savepath + "frame000" + str(i) + ".png", image2)
    elif i < 100:
        cv2.imwrite(savepath + "frame00" + str(i) + ".png", image2)
    elif i < 1000:
        cv2.imwrite(savepath + "frame0" + str(i) + ".png", image2)
    else:
        cv2.imwrite(savepath + "frame" + str(i) + ".png", image2)
    if i == 10 or i == 20:
        image3 = image.copy()
        for x in range(1, width - 1):
            for y in range(1, height - 1):
                if array[y][x] > 0:
                    image3[y - 1, x - 1][0] = 0
                    image3[y - 1, x - 1][1] = 0
                    image3[y - 1, x - 1][2] = 255
        cv2.imwrite(savepath + "image" + str(i) + ".png", image3)


if __name__ == "__main__":
    main()


197
4067.77080074848
0.00018437616983287214
0
1
4575.157595067208
0.00016392877937333275
0
2
4726.812594270842
0.00015866929036049396
0
3
4886.564912702599
0.00015348204994686124
0
4
4911.634397308284
0.00015269866185704323
1
5
5112.636227608611
0.00014669535766107216
2
6
5149.38173075149
0.0001456485534022638
0
7
5159.485480304551
0.00014536333184054807
0
8
5176.401896144249
0.00014488828631305717
1
9
4987.031844592182
0.00015039005632444118
2
10
5008.569050587705
0.00014974336830037217


KeyboardInterrupt: 